## Aufgabe 12.1.1 – MQTT-Füllstandssimulation

Die Füllstände der drei Dispenser werden alle **10 Sekunden** über MQTT an den angegebenen Broker gesendet.
Da keine echten Sensorwerte vorliegen, werden simulierte Werte verwendet, obwohl alles für die echten Sensorwerte vorbereitet wurde.
Alle Nachrichten werden mit `retain` gesendet.

### MQTT-Übersicht

```plaintext
aut/PaToMa/$groupsname                  ➔  PaToMa
aut/PaToMa/names                        ➔  Neuner, Czermak, Eckstein
aut/PaToMa/FillLevels/Dispenser_Red     ➔  <REAL>
aut/PaToMa/FillLevels/Dispenser_Green   ➔  <REAL>
aut/PaToMa/FillLevels/Dispenser_Blue    ➔  <REAL>
aut/PaToMa/FillLevels/$$unit            ➔  mm
```

### MQTT Explorer Ansicht

![MQTT Explorer](../images/MQTT_Explorer.png)

### Codeausschnitt (TwinCAT 3)

In [ ]:
FUNCTION_BLOCK FB_Disp_FillLevel_Simulation
VAR_INPUT
	iSensor : UINT;	// 0 - 65535 (16 bit)
	(* zufälliger Füllstand *)
	iOffset	: UINT;
END_VAR
VAR_OUTPUT
	rDistance_mm       	: REAL;   // Abstand Sensor – Granulat
	rFillLevel_mm    	: REAL;   // 0 – rMaxFilllevel
	rFillLevel_Percent	: REAL;   // 0 – 100%
END_VAR
VAR
	rMinDist_mm		: REAL := 30.0;   // Totzone des Sensors (lt. Datenblatt)
	rMaxDist_mm		: REAL := 350.0;  // maximal messbarer Abstand
	(* zufälliger Füllstand *)
	rRandomPercent	: REAL;
END_VAR

(*
	ADC-Werte der Füllstandssensoren werden in der HMI-Simulation nicht ausgegeben,
	deswegen werden statt der realen Implementierung (ausgeklammert), Zufallswerte für die Füllstände ermittelt 

// Abstand vom Sensor zur Oberfläche
rDistance_mm := FUN_Scale(iSensor, 65535, 0, rMaxDist_mm, rMinDist_mm);

// Füllhöhe = maximale Füllhöhe – Abstand (Sensor)
rFillLevel_mm := IFC_HW.rMaxFillLevel - rDistance_mm;

// prozentualer Füllstand
rFillLevel_percent := (rFillLevel_mm / IFC_HW.rMaxFillLevel) * 100.0;

IF rFillLevel_percent > 100.0 THEN
  rFillLevel_percent := 100.0;
ELSIF rFillLevel_percent < 0.0 THEN
  rFillLevel_percent := 0.0;
END_IF;
*)

rRandomPercent := TO_REAL((TIME_TO_UDINT(TIME()) + iOffset) MOD 10000) / 100;
rFillLevel_Percent := 100.0 - rRandomPercent;
rFillLevel_mm := (rFillLevel_Percent / 100.0) * IFC_HW.rMaxFillLevel;

----------------------------------------------------------------------------------------------------

PROGRAM MQTT_Client
VAR
	fbmqttclient : FB_IotMqttClient;	// MQTT-Client Instanz
	bInitDone : BOOL := FALSE;			// Flag für einmalige Initialnachrichten
	
	// MQTT Topics
	sTopicGroupsname	: STRING(255) := '$$groupsname'; 
	sTopicNames			: STRING(255) := 'names';
	sTopicDispRed		: STRING(255) := 'FillLevels/Dispenser_Red'; 
	sTopicDispGreen		: STRING(255) := 'FillLevels/Dispenser_Green'; 
	sTopicDispBlue		: STRING(255) := 'FillLevels/Dispenser_Blue';
	sTopicUnit			: STRING(255) := 'FillLevels/$$unit'; 
	
	// Nachrichteninhalte
	sMessageGroupsname	: STRING(255) := 'PaToMa';
	sMessageNames		: STRING(255) := 'Neuner, Czermak, Eckstein';
	sMessageUnit		: STRING(255) := 'mm';
	sMessageToPublish	: STRING(255); // Message Buffer
	
	// Zeitgeber für zyklisches Senden der Füllstände (alle 10s)
	tmrSendMessageInterval: TON := (PT := T#10S);
	
	// Offsets für Zufallsfüllstände pro Dispenser (damit nicht alle Werte ident sind)
	iOffset_Disp_1	: UINT	:= 0;
	iOffset_Disp_2	: UINT	:= 1234;
	iOffset_Disp_3	: UINT	:= 5678;
	
	// Simulierte Füllstände in mm
	FillLevel_Disp_1	: REAL;
	FillLevel_Disp_2	: REAL;
	FillLevel_Disp_3	: REAL;
	
	// Bausteine für Füllstandsermittlung
	Fb_FillLevel_Disp_1 : FB_Disp_FillLevel_Simulation;
	Fb_FillLevel_Disp_2 : FB_Disp_FillLevel_Simulation;
	Fb_FillLevel_Disp_3 : FB_Disp_FillLevel_Simulation;
END_VAR

// MQTT Konfiguration
fbMqttClient.sClientId := 'Publishing PLC';
fbMqttClient.sHostName := '158.180.44.197';
fbMqttClient.nHostPort := 1883;
fbMqttClient.sTopicPrefix := 'aut/PaToMa/';
fbMqttClient.sUserName := 'bobm';
fbMqttClient.sUserPassword := 'letmein';

fbmqttclient.Execute(TRUE);

// Füllstände simulieren
Fb_FillLevel_Disp_1(iOffset:= iOffset_Disp_1, rFillLevel_mm=> FillLevel_Disp_1);
Fb_FillLevel_Disp_2(iOffset:= iOffset_Disp_2, rFillLevel_mm=> FillLevel_Disp_2);
Fb_FillLevel_Disp_3(iOffset:= iOffset_Disp_3, rFillLevel_mm=> FillLevel_Disp_3);

IF fbMqttClient.bConnected THEN
    tmrSendMessageInterval(IN := TRUE); // Timer starten
	
	// Metadaten einmalig bei Start senden
	IF NOT bInitDone THEN
		// Groupsname
		fbMqttclient.Publish(
            sTopic := sTopicGroupsname,
            pPayload := ADR(sMessageGroupsname),
            nPayloadSize := LEN(sMessageGroupsname),
            eQoS := TcIotMqttQos.AtMostOnceDelivery,
            bRetain := TRUE,
            bQueue := FALSE);
		// Names
		fbMqttclient.Publish(
            sTopic := sTopicNames,
            pPayload := ADR(sMessageNames),
            nPayloadSize := LEN(sMessageNames),
            eQoS := TcIotMqttQos.AtMostOnceDelivery,
            bRetain := TRUE,
            bQueue := FALSE);
		// Unit 
		fbMqttclient.Publish(
            sTopic := sTopicUnit,
            pPayload := ADR(sMessageUnit),
            nPayloadSize := LEN(sMessageUnit),
            eQoS := TcIotMqttQos.AtMostOnceDelivery,
            bRetain := TRUE,
            bQueue := FALSE);
		
		bInitDone := TRUE;
	END_IF
	
	// zyklisches Senden der simulierten Füllstände
    IF tmrSendMessageInterval.Q THEN
        tmrSendMessageInterval(IN := FALSE); // Timer Reset
		
		// Dispenser Red
		sMessageToPublish := REAL_TO_STRING(FillLevel_Disp_1);
		fbMqttclient.Publish(
            sTopic := sTopicDispRed,
            pPayload := ADR(sMessageToPublish),
            nPayloadSize := LEN(sMessageToPublish),
            eQoS := TcIotMqttQos.AtMostOnceDelivery,
            bRetain := TRUE,
            bQueue := FALSE);
			
		// Dispenser Green
		sMessageToPublish := REAL_TO_STRING(FillLevel_Disp_2);
		fbMqttclient.Publish(
            sTopic := sTopicDispGreen,
            pPayload := ADR(sMessageToPublish),
            nPayloadSize := LEN(sMessageToPublish),
            eQoS := TcIotMqttQos.AtMostOnceDelivery,
            bRetain := TRUE,
            bQueue := FALSE);
		
		// Dispenser Blue
		sMessageToPublish := REAL_TO_STRING(FillLevel_Disp_3);
		fbMqttclient.Publish(
            sTopic := sTopicDispBlue,
            pPayload := ADR(sMessageToPublish),
            nPayloadSize := LEN(sMessageToPublish),
            eQoS := TcIotMqttQos.AtMostOnceDelivery,
            bRetain := TRUE,
            bQueue := FALSE);
    END_IF
END_IF

> Alle Metadaten (`groupsname`, `names`, `unit`) werden nur einmalig beim Systemstart übertragen.  
> Die Füllstände werden periodisch alle 10 s aktualisiert.